In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [13]:

#------------------------------------------------------------------------------
# THIS DATA IS NOT CONTINUED FROM THE PREV LAB FOR SIMPLICITY
#------------------------------------------------------------------------------

filtered_loans = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/filtered_loans_2007_dropTwoVar.csv')
print(filtered_loans.shape)
filtered_loans.head()

(39239, 22)


,loan_amnt,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies,fico_average
0,5000,36 months,162.87,B,10+ years,RENT,24000.0,Verified,1,credit_card,...,0,Jan-85,1,3,0,13648,0.837,9,0.0,737
1,2500,60 months,59.83,C,< 1 year,RENT,30000.0,Source Verified,0,car,...,0,Apr-99,5,3,0,1687,0.094,4,0.0,742
2,2400,36 months,84.33,C,10+ years,RENT,12252.0,Not Verified,1,small_business,...,0,Nov-01,2,2,0,2956,0.985,10,0.0,737
3,10000,36 months,339.31,C,10+ years,RENT,49200.0,Source Verified,1,other,...,0,Feb-96,1,10,0,5598,0.210,37,0.0,692
4,5000,36 months,156.46,A,3 years,RENT,36000.0,Source Verified,1,wedding,...,0,Nov-04,3,9,0,7963,0.283,12,0.0,732


In [5]:
null_counts = filtered_loans.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
loan_amnt                  0
term                       0
installment                0
grade                      0
emp_length              1057
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
pub_rec_bankruptcies     697
fico_average               0
dtype: int64


In [14]:
filtered_loans.dtypes

loan_amnt                 int64
term                     object
installment             float64
grade                    object
emp_length               object
home_ownership           object
annual_inc              float64
verification_status      object
loan_status               int64
purpose                  object
addr_state               object
dti                     float64
delinq_2yrs               int64
earliest_cr_line         object
inq_last_6mths            int64
open_acc                  int64
pub_rec                   int64
revol_bal                 int64
revol_util              float64
total_acc                 int64
pub_rec_bankruptcies    float64
fico_average              int64
dtype: object

In [15]:
# 1. Select numeric columns only
numeric_cols = filtered_loans.select_dtypes(include=['number']).columns.tolist()
categorical_cols = [col for col in filtered_loans.columns if col not in numeric_cols]

# 2. Define mean imputation transformer for numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])
# 3. Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'  # keep non-numeric columns unchanged
)

# 4. Apply the pipeline
filtered_loans_imputed = preprocessor.fit_transform(filtered_loans)

# 5. Reconstruct the DataFrame
filtered_loans_imputed = pd.DataFrame(filtered_loans_imputed, columns=numeric_cols + categorical_cols)
filtered_loans_imputed = filtered_loans_imputed[filtered_loans.columns.tolist()]

In [16]:
filtered_loans_imputed

,loan_amnt,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies,fico_average
0,5000.0,36 months,162.87,B,10+ years,RENT,24000.0,Verified,1.0,credit_card,...,0.0,Jan-85,1.0,3.0,0.0,13648.0,0.837,9.0,0.0,737.0
1,2500.0,60 months,59.83,C,< 1 year,RENT,30000.0,Source Verified,0.0,car,...,0.0,Apr-99,5.0,3.0,0.0,1687.0,0.094,4.0,0.0,742.0
2,2400.0,36 months,84.33,C,10+ years,RENT,12252.0,Not Verified,1.0,small_business,...,0.0,Nov-01,2.0,2.0,0.0,2956.0,0.985,10.0,0.0,737.0
3,10000.0,36 months,339.31,C,10+ years,RENT,49200.0,Source Verified,1.0,other,...,0.0,Feb-96,1.0,10.0,0.0,5598.0,0.21,37.0,0.0,692.0
4,5000.0,36 months,156.46,A,3 years,RENT,36000.0,Source Verified,1.0,wedding,...,0.0,Nov-04,3.0,9.0,0.0,7963.0,0.283,12.0,0.0,732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39234,2500.0,36 months,78.42,A,4 years,MORTGAGE,110000.0,Not Verified,1.0,home_improvement,...,0.0,Nov-90,0.0,13.0,0.0,7274.0,0.131,40.0,0.043459,762.0
39235,8500.0,36 months,275.38,C,3 years,RENT,18000.0,Not Verified,1.0,credit_card,...,1.0,Dec-86,1.0,6.0,0.0,8847.0,0.269,9.0,0.043459,692.0
39236,5000.0,36 months,156.84,A,< 1 year,MORTGAGE,100000.0,Not Verified,1.0,debt_consolidation,...,0.0,Oct-98,0.0,11.0,0.0,9698.0,0.194,20.0,0.043459,742.0
39237,5000.0,36 months,155.38,A,< 1 year,MORTGAGE,200000.0,Not Verified,1.0,other,...,0.0,Nov-88,0.0,17.0,0.0,85607.0,0.007,26.0,0.043459,812.0


In [39]:
null_counts = filtered_loans_imputed.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
loan_amnt                  0
term                       0
installment                0
grade                      0
emp_length              1057
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                 0
total_acc                  0
pub_rec_bankruptcies       0
fico_average               0
dtype: int64


In [40]:
# 1. Identify column types
numeric_cols = filtered_loans.select_dtypes(include=['number']).columns.tolist()
categorical_cols = [col for col in filtered_loans.columns if col not in numeric_cols]

# 2. Define transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# 3. ColumnTransformer for both
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# 4. Fit and transform
filtered_loans_imputed = preprocessor.fit_transform(filtered_loans)

# 5. Reconstruct DataFrame with original column order
filtered_loans_imputed = pd.DataFrame(filtered_loans_imputed, columns=numeric_cols + categorical_cols)
filtered_loans_imputed = filtered_loans_imputed[filtered_loans.columns.tolist()]

In [41]:
null_counts = filtered_loans_imputed.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
loan_amnt               0
term                    0
installment             0
grade                   0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
loan_status             0
purpose                 0
addr_state              0
dti                     0
delinq_2yrs             0
earliest_cr_line        0
inq_last_6mths          0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
pub_rec_bankruptcies    0
fico_average            0
dtype: int64
